In [ ]:
# ============ CELL 1: Setup ============
!pip install sentence-transformers chromadb torch

from google.colab import drive
drive.mount('/content/drive')

import torch
print(f"GPU available: {torch.cuda.is_available()}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.4 MB/s e

In [ ]:

# ============ CELL 2: OPEA Embedding Microservice ============
import json
from sentence_transformers import SentenceTransformer
from typing import List, Dict
import numpy as np
from tqdm import tqdm

class OPEAEmbeddingService:
    """OPEA Microservice for Text Embeddings"""

    def __init__(self, model_name: str = "intfloat/multilingual-e5-large"):
        print(f"Loading embedding model: {model_name}")
        self.model = SentenceTransformer(model_name)
        self.embedding_dim = self.model.get_sentence_embedding_dimension()
        print(f"✓ Model loaded (dimension: {self.embedding_dim})")

    def generate_embeddings(self, chunks: List[Dict], batch_size: int = 32) -> List[Dict]:
        """Generate embeddings for chunks"""
        texts = [chunk['text'] for chunk in chunks]

        # Generate embeddings in batches
        all_embeddings = []

        for i in tqdm(range(0, len(texts), batch_size), desc="Generating embeddings"):
            batch_texts = texts[i:i+batch_size]
            batch_embeddings = self.model.encode(
                batch_texts,
                show_progress_bar=False,
                convert_to_numpy=True
            )
            all_embeddings.extend(batch_embeddings)

        # Add embeddings to chunks
        for chunk, embedding in zip(chunks, all_embeddings):
            chunk['embedding'] = embedding.tolist()

        return chunks

    def save_embedded_chunks(self, chunks: List[Dict], output_path: str):
        """Save chunks with embeddings"""
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(chunks, f, ensure_ascii=False, indent=2)

        print(f"✓ Saved {len(chunks)} embedded chunks to {output_path}")

# ============ CELL 3: Generate Embeddings ============
embedding_service = OPEAEmbeddingService("intfloat/multilingual-e5-large")

chunk_files = [
    '/content/drive/MyDrive/ncert_processed/6_science_english_chunks.json',
    '/content/drive/MyDrive/ncert_processed/6_science_hindi_chunks.json',
    # Add more files
]

for chunk_file in chunk_files:
    print(f"\nProcessing {chunk_file}...")

    # Load chunks
    with open(chunk_file, 'r') as f:
        chunks = json.load(f)

    # Generate embeddings
    embedded_chunks = embedding_service.generate_embeddings(chunks, batch_size=32)

    # Save
    output_file = chunk_file.replace('_chunks.json', '_embedded.json')
    embedding_service.save_embedded_chunks(embedded_chunks, output_file)

print("\n✅ Embedding generation completed!")

Loading embedding model: intfloat/multilingual-e5-large


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

✓ Model loaded (dimension: 1024)

Processing /content/drive/MyDrive/ncert_processed/6_science_english_chunks.json...


Generating embeddings: 100%|██████████| 6/6 [00:14<00:00,  2.37s/it]


✓ Saved 162 embedded chunks to /content/drive/MyDrive/ncert_processed/6_science_english_embedded.json

Processing /content/drive/MyDrive/ncert_processed/6_science_hindi_chunks.json...


Generating embeddings: 100%|██████████| 6/6 [00:15<00:00,  2.62s/it]


✓ Saved 181 embedded chunks to /content/drive/MyDrive/ncert_processed/6_science_hindi_embedded.json

✅ Embedding generation completed!
